In [1]:
# 取得資料
!wget http://bigdata.fychao.info/data/newTTS.7z

# 安裝7zip
!apt update; apt install p7zip-full

# 升級pip
!pip install --upgrade pip

--2019-05-11 05:44:49--  http://bigdata.fychao.info/data/newTTS.7z
Resolving bigdata.fychao.info (bigdata.fychao.info)... 140.135.77.85
Connecting to bigdata.fychao.info (bigdata.fychao.info)|140.135.77.85|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2182132685 (2.0G) [application/x-7z-compressed]
Saving to: ‘newTTS.7z’

newTTS.7z           100%[===================>]   2.03G  27.0MB/s    in 2m 50s  

2019-05-11 05:47:44 (12.2 MB/s) - ‘newTTS.7z’ saved [2182132685/2182132685]



# 解壓縮檔案
!7z x newTTS.7z

In [4]:
# 以下為 WINDOWS 版本執行
# 多執行緒
!pip install joblib

In [1]:
import os
import re
from joblib import Parallel, delayed
from math import sqrt

In [ ]:
all_mail = {}
all_mail_code = set()
with open("D:\\newTTS1.csv", 'r') as mfile:
    for line in mfile:
        (mail_code, status, post_code, mtime) = line.strip().split(",")
        
        if not mail_code in all_mail_code:
            all_mail_code.add(mail_code)
            all_mail[mail_code] = []
          
        all_mail[mail_code].append((status, post_code, mtime))

In [24]:
all_mail_code = list(all_mail_code)

In [25]:
for mail_code in all_mail_code[:3]:
    print(mail_code, all_mail[mail_code])

00174350105816 [('Y4', '200600', '2018-01-02 09:13:52'), ('I4', '200600', '2018-01-02 13:26:48')]
95636073000410612003 [('A1', '730004', '2018-01-02 16:26:00'), ('Z4', '704672', '2018-01-02 22:42:08'), ('Y4', '600006', '2018-01-03 09:23:57'), ('Y4', '600006', '2018-01-03 11:05:13'), ('I4', '600006', '2018-01-03 14:17:49')]
02625390103317 [('Y4', '800584', '2018-01-02 08:42:07'), ('H4', '800584', '2018-01-02 14:25:22'), ('Y1', '800584', '2018-01-02 17:02:01')]


In [28]:
def makeNodeEdge(mail_proc):
    results = []
    for idx in range(len(mail_proc) - 1):
        (frm_status, frm_post_code, frm_mtime) = mail_proc[idx]
        (to_status, to_post_code, to_mtime) = mail_proc[idx + 1]
        results.append((frm_status, to_status, frm_post_code, to_post_code, frm_mtime, to_mtime))
    return results
makeNodeEdge(all_mail["95636073000410612003"])

[('A1',
  'Z4',
  '730004',
  '704672',
  '2018-01-02 16:26:00',
  '2018-01-02 22:42:08'),
 ('Z4',
  'Y4',
  '704672',
  '600006',
  '2018-01-02 22:42:08',
  '2018-01-03 09:23:57'),
 ('Y4',
  'Y4',
  '600006',
  '600006',
  '2018-01-03 09:23:57',
  '2018-01-03 11:05:13'),
 ('Y4',
  'I4',
  '600006',
  '600006',
  '2018-01-03 11:05:13',
  '2018-01-03 14:17:49')]

In [47]:
all_edges = Parallel(n_jobs = -1)(delayed(makeNodeEdge)(all_mail[mail_code]) for mail_code in all_mail_code)

In [49]:
print(len(all_edges))

4419944


In [51]:
# 由於是list of list，所以要把資料打平
flat_all_edges = [y for x in all_edges for y in x]

In [53]:
flat_all_edges[:3]

[('Y4',
  'I4',
  '200600',
  '200600',
  '2018-01-02 09:13:52',
  '2018-01-02 13:26:48'),
 ('A1',
  'Z4',
  '730004',
  '704672',
  '2018-01-02 16:26:00',
  '2018-01-02 22:42:08'),
 ('Z4',
  'Y4',
  '704672',
  '600006',
  '2018-01-02 22:42:08',
  '2018-01-03 09:23:57')]

In [54]:
import pandas as pd

In [57]:
df = pd.DataFrame(flat_all_edges)
df.columns = ["frm_status", "to_status", "frm_post_code", "to_post_code", "frm_mtime", "to_mtime"]

In [58]:
df['frm_mtime'] = pd.to_datetime(df['frm_mtime'])
df['to_mtime'] = pd.to_datetime(df['to_mtime'])
df['diff_mtime'] = df['to_mtime'] - df['frm_mtime']

In [59]:
df.head(5)

,frm_status,to_status,frm_post_code,to_post_code,frm_mtime,to_mtime,diff_mtime
0,Y4,I4,200600,200600,2018-01-02 09:13:52,2018-01-02 13:26:48,04:12:56
1,A1,Z4,730004,704672,2018-01-02 16:26:00,2018-01-02 22:42:08,06:16:08
2,Z4,Y4,704672,600006,2018-01-02 22:42:08,2018-01-03 09:23:57,10:41:49
3,Y4,Y4,600006,600006,2018-01-03 09:23:57,2018-01-03 11:05:13,01:41:16
4,Y4,I4,600006,600006,2018-01-03 11:05:13,2018-01-03 14:17:49,03:12:36


In [60]:
df['frm_init'] = df['frm_status'].str.slice(0,1)
df['to_init'] = df['to_status'].str.slice(0,1)

In [61]:
df['frm_code'] = df['frm_post_code'] + '-' + df['frm_init']
df['to_code'] = df['to_post_code'] + '-' + df['to_init']

In [65]:
ndf = df[['frm_init', 'to_init', 'frm_code', 'to_code', 'diff_mtime']]

In [67]:
ndf.set_index(['frm_init', 'to_init'], inplace = True)
ndf.sort_index()

frm_code   to_code      diff_mtime
frm_init to_init                                    
A        A        100096-A  100096-A 0 days 01:36:00
         A        400016-A  400016-A 0 days 06:15:00
         A        600015-A  600015-A 0 days 01:45:00
         A        100451-A  100451-A 0 days 01:09:00
         A        400451-A  400451-A 0 days 00:01:00
         A        400069-A  400069-A 0 days 06:49:00
         A        830051-A  830051-A 0 days 00:11:00
         A        330035-A  330035-A 0 days 00:07:00
         A        100451-A  100451-A 0 days 01:09:00
         A        420001-A  420001-A 0 days 04:44:00
         A        800058-A  800058-A 0 days 00:56:00
         A        330035-A  330035-A 0 days 00:07:00
         A        420001-A  420001-A 0 days 04:44:00
         A        100068-A  100068-A 0 days 01:12:00
         A        241021-A  241021-A 0 days 00:01:00
         A        100107-A  100107-A 0 days 00:02:00
         A        100096-A  100096-A 0 days 01:36:00
         A        704011-A  704011-A 0 days 00:05:00
         A        800058-A  800058-A 0 days 00:56:00
         A        241046-A  241046-A 0 days 00:02:00
         A        241046-A  241046-A 0 days 00:02:00
         A        800046-A  800046-A 1 days 00:09:00
         A        100081-A  100081-A 0 days 23:51:00
         A        100055-A  100055-A 0 days 23:36:00
         A        241023-A  241023-A 0 days 00:03:00
         A        220064-A  220064-A 0 days 00:03:00
         A        730010-A  730010-A 0 days 05:59:00
         A        400016-A  400016-A 0 days 01:37:00
         A        100046-A  100046-A 0 days 00:04:00
         A        100451-A  100451-A 0 days 01:09:00
...                    ...       ...             ...
Z        Z        400672-Z  400047-Z 0 days 10:06:00
         Z        330672-Z  950580-Z 0 days 14:16:02
         Z        400672-Z  420600-Z 0 days 07:23:44
         Z        400672-Z  500001-Z 0 days 10:21:42
         Z        091813-Z  400672-Z 0 days 05:29:00
         Z        330672-Z  400672-Z 0 days 06:15:27
         Z        400672-Z  091814-Z 0 days 11:12:36
         Z        100572-Z  091813-Z 0 days 09:58:08
         Z        091813-Z  400672-Z 0 days 05:34:00
         Z        950022-Z  950580-Z 0 days 04:26:20
         Z        400672-Z  091814-Z 0 days 06:21:29
         Z        100572-Z  091813-Z 0 days 06:36:30
         Z        330672-Z  400672-Z 0 days 04:42:24
         Z        400672-Z  091813-Z 0 days 09:15:54
         Z        400672-Z  400584-Z 0 days 02:59:39
         Z        091813-Z  704584-Z 0 days 06:53:31
         Z        800672-Z  800672-Z 0 days 00:38:19
         Z        420033-Z  400672-Z 0 days 07:13:38
         Z        400672-Z  420600-Z 0 days 06:56:10
         Z        330672-Z  400672-Z 0 days 06:20:38
         Z        400672-Z  420032-Z 0 days 06:13:29
         Z        400672-Z  970582-Z 0 days 16:42:53
         Z        400672-Z  704672-Z 0 days 06:42:47
         Z        600584-Z  704672-Z 0 days 08:39:46
         Z        330672-Z  091814-Z 0 days 06:11:55
         Z        800672-Z  091814-Z 0 days 10:24:45
         Z        330672-Z  320040-Z 0 days 12:14:16
         Z        320040-Z  330672-Z 0 days 13:20:56
         Z        830031-Z  800672-Z 0 days 05:13:16
         Z        091814-Z  400672-Z 0 days 07:01:09

[5580056 rows x 3 columns]

In [69]:
ndf.loc[('A', 'A')].diff_mtime.mean()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


Timedelta('0 days 03:26:49.707753')

In [98]:
mean_time = []
for frm_scode in sorted(df.frm_init.unique()):
    for to_scode in sorted(df.to_init.unique()):
        if (frm_scode, to_scode) in ndf.index:
            mean_time.append(((frm_scode, to_scode),
                              len(ndf.loc[(frm_scode, to_scode)]),
                              ndf.loc[(frm_scode, to_scode)].diff_mtime.mean(),
                              ndf.loc[(frm_scode, to_scode)].diff_mtime.var()
                             ))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  


In [99]:
mean_df = pd.DataFrame(mean_time)

In [100]:
mean_df.columns = ['frm_to', 'cntr', 'diff_time', 'var']

In [105]:
mean_df['percent_cntr'] = (100*mean_df['cntr']) / mean_df['cntr'].sum()

In [106]:
mean_df.sort_values("percent_cntr", ascending = False)

,frm_to,cntr,diff_time,var,percent_cntr
110,"(Y, I)",2267546,06:15:36.843593,7.308466e+25,40.636617
11,"(A, Z)",1177466,07:24:03.825907,1.523006e+26,21.101329
128,"(Z, Y)",925612,09:12:28.374041,9.711593e+25,16.587862
109,"(Y, H)",403366,05:10:43.740711,2.083188e+25,7.228709
129,"(Z, Z)",163519,07:38:58.001308,1.972558e+26,2.930419
31,"(H, Y)",132682,14:31:48.703079,6.112613e+26,2.377790
119,"(Z, G)",112122,07:28:14.870462,1.473963e+27,2.009335
32,"(H, Z)",97892,04:51:44.618201,3.303743e+26,1.754319
15,"(G, I)",83112,05:27:49.732168,9.391983e+26,1.489447
74,"(T, Z)",32117,03:24:59.735093,1.200783e+26,0.575568
